# Time Stamp Extract

Brief 1-2 sentence description of notebook.

TODO: Supplement the description
- Notebook that extracts the timestamps and gets the time that tones played

In [1]:
# Imports of all used packages and libraries
import sys
import os
import git
import glob
from collections import defaultdict

In [2]:
git_repo = git.Repo(".", search_parent_directories=True)
git_root = git_repo.git.rev_parse("--show-toplevel")

In [3]:
git_root

'/nancy/user/riwata/projects/reward_comp_ext'

In [4]:
sys.path.insert(0, os.path.join(git_root, 'src'))

In [5]:
# Imports of all used packages and libraries
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
import spikeinterface.extractors as se
import spikeinterface.preprocessing as sp

In [7]:
import utilities.helper
import trodes.read_exported

# Functions

In [8]:
# Function to filter rows where start/stop range overlap with the specified range
def filter_overlapping_ranges(frame_ranges, start=0, stop=np.inf):
    # Check for overlap
    overlaps = np.logical_or(np.logical_and(frame_ranges[:,0] < stop, frame_ranges[:,0] >= start),
                             np.logical_and(frame_ranges[:,1] <= stop, frame_ranges[:,1] > start))
    return frame_ranges[overlaps]

## Inputs & Data

- Explanation of each input and where it comes from.

Inputs and Required data loading
- input variable names are in all caps snake case
- Whenever an input changes or is used for processing 
- The variables are all lower in snake case

In [ ]:
# Path of the directory that contains the Spike Gadgets recording and the exported timestamp files
# Exported with this tool https://docs.spikegadgets.com/en/latest/basic/ExportFunctions.html
# Export these files:
    # -raw – Continuous raw band export.
    # -dio – Digital IO channel state change export.
    # -analogio – Continuous analog IO export.
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

os.makedirs(OUTPUT_DIR, exist_ok=True)
OUTPUT_PREFIX = "rce_pilot_2_both_rewarded"

In [10]:
TRODES_METADATA_DF = pd.read_pickle("./proc/{}_00_trodes_metadata.pkl".format(OUTPUT_PREFIX))

## Outputs

Describe each output that the notebook creates. 

- Is it a plot or is it data?

- How valuable is the output and why is it valuable or useful?

## Other documentation

raw directory
- raw_group0.dat
    - voltage_value: Array with voltage measurement for each channel at each timestamp
- timestamps.dat
    - voltage_time_stamp: The time stamp of each voltage measurement

parent directory
- 1.videoTimeStamps.cameraHWSync
    - frame_number: Calculated by getting the index of each video time stamp tuple 
    - PosTimestamp: The time stamp of each video frame
    - HWframeCount: Unknown value. Starts at 30742 and increases by 1 for each tuple  
    - HWTimestamp: Unknown value. All zeroes
    - video_time: Calculated by dividing the frame number by the fps(frames per second) 
    - video_seconds: video_time, but rounded to seconds  	
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_PosTimestamp 	
        - filledHWframeCount 	
        - filled_frame_number 	
        - filled_video_time 	
        - filled_video_seconds 	

DIO directory
- dio_ECU_Din1.dat
    - time: The time stamp the corresponds to the DIN input
    - state: Binary state of whether there is input from DIN or not 	
    - trial_number: Calculated by adding 1 to every time there is a DIN input
    - These are filled in versions of the above collumns with the value from the most recent previous cell
        - filled_state 	
        - filled_trial_number

ss_output directory (Spike sorting with Spike interface)
- firings.npz
    - unit_id: All the units that had a spike train for the given timestamp 	
    - number_of_units: Calculated by counting the number of units that had a spike train

## Functions

- function names are short and in snake case all lowercase
- a function name should be unique but does not have to describe the function
- doc strings describe functions not function names

## Processing

Describe what is done to the data here and how inputs are manipulated to generate outputs. 

In [11]:
# As much code and as many cells as required
# includes EDA and playing with data
# GO HAM!

In [12]:
TRODES_METADATA_DF = TRODES_METADATA_DF.drop(columns=["current_subject", "video_timestamps", "recording", "session_path"], errors="ignore")
TRODES_METADATA_DF = TRODES_METADATA_DF.drop_duplicates(subset=["video_name"])

In [13]:
exploded_TRODES_METADATA_DF = TRODES_METADATA_DF.explode(column = ["tone_frames", "tone_timestamps"])

In [14]:
exploded_TRODES_METADATA_DF

,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240327_141822_comp_both_subj_5-2_and_5-3,"[1, 26]","[[1, 26], [29421, 29445], [30032, 30049], [301...","[[1, 26], [32, 35], [36, 39], [87, 89], [90, 1...",20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,"[0, 25621]","[[0, 25621], [29432589, 29451591], [29926394, ...","[[0, 25621], [31621, 34424], [34821, 37424], [..."
0,20240327_141822_comp_both_subj_5-2_and_5-3,"[1224, 1423]","[[1, 26], [29421, 29445], [30032, 30049], [301...","[[1, 26], [32, 35], [36, 39], [87, 89], [90, 1...",20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,"[1226037, 1426041]","[[0, 25621], [29432589, 29451591], [29926394, ...","[[0, 25621], [31621, 34424], [34821, 37424], [..."
0,20240327_141822_comp_both_subj_5-2_and_5-3,"[3220, 3421]","[[1, 26], [29421, 29445], [30032, 30049], [301...","[[1, 26], [32, 35], [36, 39], [87, 89], [90, 1...",20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,"[3226062, 3426067]","[[0, 25621], [29432589, 29451591], [29926394, ...","[[0, 25621], [31621, 34424], [34821, 37424], [..."
0,20240327_141822_comp_both_subj_5-2_and_5-3,"[4418, 4618]","[[1, 26], [29421, 29445], [30032, 30049], [301...","[[1, 26], [32, 35], [36, 39], [87, 89], [90, 1...",20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,"[4426080, 4626082]","[[0, 25621], [29432589, 29451591], [29926394, ...","[[0, 25621], [31621, 34424], [34821, 37424], [..."
0,20240327_141822_comp_both_subj_5-2_and_5-3,"[5417, 5616]","[[1, 26], [29421, 29445], [30032, 30049], [301...","[[1, 26], [32, 35], [36, 39], [87, 89], [90, 1...",20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,"[5426092, 5626092]","[[0, 25621], [29432589, 29451591], [29926394, ...","[[0, 25621], [31621, 34424], [34821, 37424], [..."
...,...,...,...,...,...,...,...,...,...,...,...
33,20240331_164629_comp_both_subj_4-3_and_4-4,"[0, 0]","[[1, 146], [0, 0], [0, 0], [0, 0], [0, 0], [0,...","[[1, 146], [441, 441], [442, 444], [445, 445],...",20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,"[55646769, 55846771]","[[0, 145690], [31298271, 31310871], [31600675,...","[[0, 145690], [441636, 441696], [442291, 44469..."
33,20240331_164629_comp_both_subj_4-3_and_4-4,"[0, 0]","[[1, 146], [0, 0], [0, 0], [0, 0], [0, 0], [0,...","[[1, 146], [441, 441], [442, 444], [445, 445],...",20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,"[56646781, 56846783]","[[0, 145690], [31298271, 31310871], [31600675,...","[[0, 145690], [441636, 441696], [442291, 44469..."
33,20240331_164629_comp_both_subj_4-3_and_4-4,"[0, 0]","[[1, 146], [0, 0], [0, 0], [0, 0], [0, 0], [0,...","[[1, 146], [441, 441], [442, 444], [445, 445],...",20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,"[58046797, 58246800]","[[0, 145690], [31298271, 31310871], [31600675,...","[[0, 145690], [441636, 441696], [442291, 44469..."
33,20240331_164629_comp_both_subj_4-3_and_4-4,"[0, 0]","[[1, 146], [0, 0], [0, 0], [0, 0], [0, 0], [0,...","[[1, 146], [441, 441], [442, 444], [445, 445],...",20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,"[60246824, 60446827]","[[0, 145690], [31298271, 31310871], [31600675,...","[[0, 145690], [441636, 441696], [442291, 44469..."


In [15]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: np.array(x))


box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [16]:
columns_to_update = ["box_1_port_entry_frames", "box_2_port_entry_frames", "box_1_port_entry_timestamps", "box_2_port_entry_timestamps"]
tone_columns = ["tone_frames", "tone_frames", "tone_timestamps", "tone_timestamps"]

for col, tone_col in zip(columns_to_update, tone_columns):
    exploded_TRODES_METADATA_DF[col] = exploded_TRODES_METADATA_DF.apply(lambda x: filter_overlapping_ranges(x[col], x[tone_col][0], x[tone_col][1]), axis=1)

In [17]:
exploded_TRODES_METADATA_DF["tone_start_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_frame"] = exploded_TRODES_METADATA_DF["tone_frames"].apply(lambda x: x[1])

In [18]:
exploded_TRODES_METADATA_DF["tone_start_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[0])
exploded_TRODES_METADATA_DF["tone_stop_timestamp"] = exploded_TRODES_METADATA_DF["tone_timestamps"].apply(lambda x: x[1])


In [19]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF.drop(columns=["tone_frames", "tone_timestamps"], errors="ignore")

- Create a new column for start and stop

In [20]:
exploded_TRODES_METADATA_DF = exploded_TRODES_METADATA_DF[sorted(exploded_TRODES_METADATA_DF.columns, key=lambda s: s.split('_')[-1])].copy()

In [21]:
exploded_TRODES_METADATA_DF.to_pickle("./proc/{}_per_video_trial_labels.pkl".format(OUTPUT_PREFIX))

- Exporting to CSV

In [22]:
for entry_col in [col for col in exploded_TRODES_METADATA_DF.columns if "port_entry" in col]:
    print(entry_col)
    exploded_TRODES_METADATA_DF[entry_col] = exploded_TRODES_METADATA_DF[entry_col].apply(lambda x: ' '.join(map(str, x)))

box_1_port_entry_frames
box_2_port_entry_frames
box_1_port_entry_timestamps
box_2_port_entry_timestamps


In [23]:
exploded_TRODES_METADATA_DF.head()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
0,20240327_141822_comp_both_subj_5-2_and_5-3,1,26,[ 1 26],[ 1 26],20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,0,25621,[ 0 25621],[ 0 25621]
0,20240327_141822_comp_both_subj_5-2_and_5-3,1224,1423,,[1241 1242] [1246 1267] [1296 1305] [1306 1307...,20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,1226037,1426041,,[1242839 1244039] [1248639 1268639] [1297837 1...
0,20240327_141822_comp_both_subj_5-2_and_5-3,3220,3421,,[3242 3248] [3267 3280] [3287 3298] [3299 3324...,20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,3226062,3426067,,[3247865 3253062] [3272865 3285665] [3291665 3...
0,20240327_141822_comp_both_subj_5-2_and_5-3,4418,4618,,[4406 4419] [4440 4442] [4443 4445] [4446 4454...,20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,4426080,4626082,,[4413079 4426677] [4448478 4449678] [4450080 4...
0,20240327_141822_comp_both_subj_5-2_and_5-3,5417,5616,,[5432 5432] [5434 5435] [5438 5450] [5451 5453...,20240327_141822_comp_both_subj_5-2_and_5-3.1.v...,"[5.2, 5.3]",1948533,66385922,5426092,5626092,,[5441092 5442092] [5442492 5444492] [5446692 5...


In [24]:
exploded_TRODES_METADATA_DF.tail()

,session_dir,tone_start_frame,tone_stop_frame,box_1_port_entry_frames,box_2_port_entry_frames,video_name,all_subjects,first_timestamp,last_timestamp,tone_start_timestamp,tone_stop_timestamp,box_1_port_entry_timestamps,box_2_port_entry_timestamps
33,20240331_164629_comp_both_subj_4-3_and_4-4,0,0,,,20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,55646769,55846771,[55640968 55655168] [55655568 55676571] [55677...,[55767570 55978772]
33,20240331_164629_comp_both_subj_4-3_and_4-4,0,0,,,20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,56646781,56846783,[56637378 56658578] [56658981 56680181] [56680...,[56543379 56666183] [56668778 56862785]
33,20240331_164629_comp_both_subj_4-3_and_4-4,0,0,,,20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,58046797,58246800,[58009797 58111396] [58112398 58372601],
33,20240331_164629_comp_both_subj_4-3_and_4-4,0,0,,,20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,60246824,60446827,[60096422 60290625] [60292824 60305422] [60305...,[60257824 60479824]
33,20240331_164629_comp_both_subj_4-3_and_4-4,0,0,,,20240331_164629_comp_both_subj_4-3_and_4-4.2.v...,"[4.3, 4.4]",1602066,65905026,61346840,61546840,,


In [25]:
exploded_TRODES_METADATA_DF.to_csv("./proc/{}_per_video_trial_labels.csv".format(OUTPUT_PREFIX))